In [1]:
import sys
from random import randrange
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF

In [3]:
inputfp = open('ratings.train','r')

In [4]:
def initialize(inputfp):
  inputfp.seek(0)
  nlines=inputfp.readline().strip()
  line_no=0
  nusers=0
  min_user=100
  min_movie=100
  nitems=0
  #Finding the min and max users and items to form matrix of respective dimension
  while line_no<int(nlines):
    line = inputfp.readline()
    li = line.strip().split()
    if int(li[0])>nusers:
      nusers=int(li[0])
    if int(li[0])<min_user:
      min_user=int(li[0])
    if int(li[1])>nitems:
      nitems=int(li[1])
    if int(li[1])<min_movie:
      min_movie=int(li[1])
    line_no+=1
  #Creating a matrix of usersXitems dimensions and Initailizing the matrix by setting all values to -1
  matrix=[]
  for i in range(nusers+1):
    matrix.append([])
    for j in range(nitems+1):
      matrix[i].append(-1)
  return matrix

def test_train(inputfp,matrix):
  inputfp.seek(0)
  line_no=0
  count_repeat=0
  omega=[] #position of available matrix entries

  nlines=inputfp.readline().strip()

  #Filling the matrix with available user's ratings and puting the available entries positions in a list omega
  #while line_no<int(nlines):
  while line_no<55172:
    line = inputfp.readline()
    li = line.strip().split()
    if matrix[int(li[0])][int(li[1])]>=0:
      count_repeat+=1
    else:
      omega.append([int(li[0]),int(li[1])])
    matrix[int(li[0])][int(li[1])]=float(li[2])
    line_no+=1

  omega_test=[]
  while line_no<int(nlines):
    line = inputfp.readline()
    li = line.strip().split()
    omega_test.append([int(li[0]),int(li[1]),float(li[2])])
    line_no+=1
  return omega,omega_test,matrix

def movie_average_user_bias(matrix):
  movie_average=[]
  user_bias=[]
  #First find movie's average rating for each movie
  for i in range (len(matrix[0])):
    movie_avg=0
    rated_by=0
    for j in range(len(matrix)):
      if matrix[j][i]!= -1:
        movie_avg+=matrix[j][i]
        rated_by+=1
    if rated_by==0:
      movie_average.append(0)
    else:
      movie_average.append(movie_avg/rated_by)

  #Now find user's bias for each user
  for i in range(len(matrix)):
    user_b=0
    user_rated=0
    for j in range(len(matrix[0])):
      if matrix[i][j]!= -1.0:
        user_b+= (matrix[i][j]-movie_average[j])
        user_rated+=1
    if user_rated==0:
      user_bias.append(0)
    else:
      user_bias.append(user_b/user_rated)
  return movie_average, user_bias

def predicted_matrix(matrix,user_bias,movie_average):
  matrix_predicted = [row[:] for row in matrix]

  negative_values=0
  for i in range(len(matrix_predicted)):
    for j in range(len(matrix_predicted[0])):
      if matrix_predicted[i][j]==-1.0:
        matrix_predicted[i][j] = user_bias[i] + movie_average[j]
      if matrix_predicted[i][j]<0.0:
        negative_values+=1
        matrix_predicted[i][j]=0
      elif matrix_predicted[i][j]>5.0:
        matrix_predicted[i][j]=5
  return matrix_predicted

def svd_w_h (matrix_predicted):
  model = NMF(n_components=12, init='random', random_state=0)
  W = model.fit_transform(matrix_predicted)
  H = model.components_
  return W,H

def random_initialization(k):
  U = []
  for i in range(610):
    U.append([])
    for j in range(k):
      U[i].append(random.uniform(0,1))
  W = np.array(U)
  V = []
  for i in range(k):
    V.append([])
    for j in range(9724):
      V[i].append(random.uniform(0,1))
  H = np.array(V)
  return W,H

def stochastic_gradient(n,lamb,omega,omega_test):
  condition = 0
  n=0.001
  lamb = 0.05
  while condition<150:
    #print(condition)
    random.shuffle(omega)
    #Train on some entries
    prev_error_test=100000000000
    for i in omega:
      #print(i[0],i[1])
      e = matrix[i[0]][i[1]]-np.dot(W[i[0],:],H[:,i[1]])
      temp = np.array([i for i in W[i[0],:]])
      W[i[0],:] = W[i[0],:] + n*(e*H[:,i[1]]-lamb*W[i[0],:])
      #print("W",W[i[0],:])
      H[:,i[1]] = H[:,i[1]] + n*(e*temp-lamb*H[:,i[1]])
      #print("H",H[:,i[1]])

      error_test = 0

    for i in omega_test:
      error_test += (i[2] - np.dot(W[i[0],:],H[:,i[1]]))**2
    if error_test<=prev_error_test:
      prev_error_test=error_test
    else:
      print("broken at ",condition)
      break
    condition+=1
      
  error_train = 0
  error_test = 0
  #Calculate error for train
  numa =0

  #Bound the matrix above and below by 0 and 5
  for j in omega:
    if (np.dot(W[j[0],:],H[:,j[1]]) > 5):
      error_train += (matrix[j[0]][j[1]] - 5)**2
    elif (np.dot(W[j[0],:],H[:,j[1]]) < 0):
      error_train += (matrix[j[0]][j[1]] - 0)**2
    else:
      error_train += (matrix[j[0]][j[1]] - np.dot(W[j[0],:],H[:,j[1]]))**2
    
  #Calculate error for test
  numb =0
  for l in omega_test:
    if (np.dot(W[l[0],:],H[:,l[1]]) > 5):
      error_test += (l[2] - 5)**2
    elif (np.dot(W[l[0],:],H[:,l[1]]) > 5):
      error_test += (l[2] - 0)**2
    else:
      error_test += (l[2] - np.dot(W[l[0],:],H[:,l[1]]))**2

  rmse_train = math.sqrt(error_train/len(omega))
  rmse_test = math.sqrt(error_test/len(omega_test))
  return rmse_train, rmse_test


In [6]:
#Training the data for different dimensions of k-hyperparameter
for k in range(8,9):
  matrix = initialize(inputfp)
  omega,omega_test,matrix = test_train(inputfp,matrix)
  movie_average, user_bias = movie_average_user_bias(matrix)
  matrix_predicted = predicted_matrix(matrix,user_bias,movie_average)
  W,H = svd_w_h (matrix_predicted)
  #for n in np.arange(0.0005,0.01,0.001):
    #for lamb in np.arange(0.005,0.1,0.01):
  n=0.001
  lamb=0.02
  rmse_train, rmse_test = stochastic_gradient(n,lamb,omega,omega_test)
  print("k = "+str(k)+" n = "+str(n)+"lamb = "+str(lamb)+" rmse_train = "+str(rmse_train)+" rmse_test = "+str(rmse_test))

k = 8 n = 0.001lamb = 0.02 rmse_train = 0.6464861408418815 rmse_test = 1.1592007929035906


In [7]:
for k in range(10,11):
  for n in np.arange(0.0005,0.01,0.004):
    for lamb in np.arange(0.005,0.1,0.04):
      matrix = initialize(inputfp)
      omega,omega_test,matrix = test_train(inputfp,matrix)
      W,H = random_initialization(k)
      rmse_train, rmse_test = stochastic_gradient(n,lamb,omega,omega_test)
      print("k = "+str(k)+" n = "+str(n)+"lamb = "+str(lamb)+" rmse_train = "+str(rmse_train)+" rmse_test = "+str(rmse_test))

k = 10 n = 0.0005lamb = 0.005 rmse_train = 0.7301761315262886 rmse_test = 0.9154691926181275


KeyboardInterrupt: 